<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Experimental_Applications/LifeTimeMeasurement%20(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# !pip install pulsestreamer
# !pip install nidaqmx
import numpy as np
import inspect,dis
import sys
from tqdm import trange
from time import *
# import pulsestreamer
# import pprint
# from pulsestreamer import PulseStreamer,findPulseStreamers,OutputState,TriggerStart,Sequence

In [ ]:
# loading the local Pulse Streamer system
IPaddress = findPulseStreamers(search_serial='')[0][0]
pulser = PulseStreamer(IPaddress)

# loading the local NIDAQmx system
system = nidaqmx.system.System.local()
DAQ_device = system.devices['Dev1']

In [13]:
# all in nanosecond range
wait1=50; wait2=20
laserOn=1000; Tmax=3000
readOn=150
steps=5; samples=1000; averages=10
laserPort=0; readoutPort=1

timeRange=np.linspace(0,Tmax,num=steps)

pulse_args = {'wait1': wait1,'wait2': wait2,'laserOn': laserOn,'timeRange': timeRange,
              'readOn': readOn,'averages': averages,'samples': samples,'laserPort': laserPort,'readoutPort': readoutPort}

In [ ]:
def lifetime_sequence(pulse_args):  # all in nanosecond range

  delay1=pulse_args['wait1']; delay2=pulse_args['wait2']; laser_on =pulse_args['laserOn']; read_on =pulse_args['wait1']
  t_vec =pulse_args['timeRange']; averages =pulse_args['averages']; samples =pulse_args['samples']; read_on =pulse_args['readOn']
  laser_port =pulse_args['laserPort']; readout_port =pulse_args['readoutPort']

  # Checking the initial condition of Pulse Streamer
  if pulser.hasSequence()!=0:
    print('Pulse Streamer has a Sequence. Do you want to proceed?\n')
    permission = int(input('Press 0/1 : \n'))
  if permission==0:
    pass

  ## Producing the Patterns
  laser_block = [(delay2,0),(laser_on,1),(delay2+t_vec[-1],0)]
  laser_pattern = [(delay1,0)]; signal_pattern = [(delay1,0)]

  for t in range(t_vec.shape[0]-1): #neglecting the last t
    if t_vec.shape[0] > t_vec[-1]/read_on:
      print(f'Please reduce the step numbers. Maximum allowed steps: {t_vec[-1]/read_on}')
      break
    laser_pattern += laser_block
    signal_pattern += [((2*delay2+laser_on),0),(read_on,1),(delay2+t_vec[t],0),(read_on,1),(t_vec[-1]-t_vec[t]-delay2-2*read_on,0)]

  if t_vec.shape[0] > t_vec[-1]/read_on:
    pass
  laser_pattern +=[(delay1,0)]; signal_pattern +=[(delay1,0)]

  ## Producing the Sequences
  reSet = pulser.reset()
  if reSet==0:
    print('Pulse Streamer has been reset.\n')
  seq = pulser.createSequence()
  seq.setDigital(laser_port,laser_pattern) ; seq.setDigital(readout_port,signal_pattern)
  print(f'Plot of the Sequence without averaging:\n {seq.plot()}\n')
  # seq*=averages

  return seq

In [ ]:
 def lifetime(pulse_args):

     numberofpoints=samples*2*steps
     buffersamplecount=numberofpoints
     DAQ_device.reset_device()

     # Counter
     counter = nidaqmx.Task()
     ciChannel = counter.ci_channels.add_ci_count_edges_chan('/Dev1/ctr1',edge=nidaqmx.constants.Edge.RISING, initial_count=0, count_direction=nidaqmx.constants.CountDirection.COUNT_UP)

     # Trigger
     counter.triggers.pause_trigger.dig_lvl_src='/Dev1/PFI4'
     counter.triggers.pause_trigger.trig_type=nidaqmx.constants.TriggerType.DIGITAL_LEVEL
     counter.triggers.pause_trigger.dig_lvl_when=nidaqmx.constants.Level.LOW  ### ?

     # Timing
     counter.timing.cfg_samp_clk_timing(rate=1e8,source='/Dev1/PFI5',active_edge=nidaqmx.constants.Edge.FALLING,sample_mode = nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=(numberofpoints))

     # Pulse streamer Gating
     gate = 'Dev1/port0/line7'
     gate_task = nidaqmx.Task()
     gate_task.do_channels.add_do_chan(lines = gate)

     cps = []
     callback=[]

     # Counter read task
     reader = nidaqmx.stream_readers.CounterReader(counter.in_stream)
     highCount = np.zeros(buffersamplecount, dtype = np.uint32)

     def readBuffer(task_handle, every_n_samples_event_type, number_of_samples, callback_data):
         counter.in_stream.read_all_avail_samp = True
         readPixels=reader.read_many_sample_uint32(highCount, number_of_samples_per_channel=- 1, timeout=10.0)   # default timeout: 10s
         cps.extend(highCount)
         callback.extend([1])
         return 0

     time.sleep(0.01)
     # Read after filling the buffer with given number of samples
     counter.register_every_n_samples_acquired_into_buffer_event(buffersamplecount,readBuffer) #after every pixel it will trigger the callback

     # Start tasks (digital output will be triggered by analog output)  ### ?
     print("starting DAQ")

     #Adding infinite loop
     t=0
     run=0
     data=[]
     finaldata=[]
     print("callback number in beginning:",len(callback))

     for i in trange(averages):
         print(run)
         time.sleep(0.05)
         #counter.control(nidaqmx.constants.TaskMode.TASK_RESERVE)
         time.sleep(0.05)
         counter.start()
         seq=lifetime_sequence(pulse_args)
        #  pulser.setTrigger(start=pulser.TriggerStart.HARDWARE_RISING,rearm=pulser.TriggerRearm.AUTO)   ### ?

         seq_num=0
         time_start=time.time()

         for s in seq:    ### ?

             t1=len(callback)
             #s.plot()
             seq_num=seq_num+1

             ## Checking the Conditions
             if pulser.hasSequence()!=0:
               print('Pulse Streamer has no Sequence uploaded!')
               pass
             if seq.isEmpty()==0:
               print('Sequence to Pulse Streamer is empty!')
               pass
             if seq.getDuration()%8==0:
               print("Sequence duration is not multiple of 8ns!\n Please check by 'seq.get_pad()' method.")
               pass

             ## Performing the Streaming
             pulser.stream(seq,n_runs=samples,final=([],0,0))
            #  ps.stream(s, n_runs=samples)
             if pulser.hasFinished()==1:
               print('Streaming has been finished.\n')
             reSet = pulser.reset()
             if reSet==0:
               print('Pulse Streamer has been reset after Streaming.\n')
             #print(seq_num)

             time_start=time.time()
             gate_task.write(True)
             while len(callback)==t1:
                 time.sleep(0.005)
             time_end=time.time()
             gate_task.write(False)

         print('time taken for on stream of all sequences', time_end-time_start)
         print("callback number average end:",len(callback))
         run=run+1
         counter.control(nidaqmx.constants.TaskMode.TASK_UNRESERVE)

     #data.append(np.mean(temp))
     print(seq_num)
     data=datainterpretdaq_pulsed_unreserve(cps,numberofpoints)
     counter.close()
     gate_task.close()
     return data

 def datainterpretdaq_pulsed_unreserve(combinedcounts,samples_in_one_average):
     combinedcountarray=np.array(combinedcounts)
     averagecount=int(len(combinedcounts)/samples_in_one_average)
     print("crosscheck number of averges=",averagecount)
     combinedcountmatrix=np.reshape(combinedcountarray,(averagecount,samples_in_one_average))
     ca=np.delete(combinedcountmatrix, -1, 1)
     z = np.zeros(averagecount, dtype=int)
     ca2=np.hstack((z[:, np.newaxis], ca))
     correctedcounts=np.subtract(combinedcountmatrix,ca2)
     averagedata=np.mean(correctedcounts,axis=0)
     return averagedata

(array([   0.,  830., 1660., 2490., 3320.]), 1930)

In [ ]:
lifetime(pulse_args)